In [25]:
import sys; sys.path.append('..')
from osp import *

In [26]:
idx,data = next(STASH_POS_COUNTS.items())
# data
idx

'phil/10.2307/20140633__02'

In [27]:
def get_pos_counts(ids):
    ids=set(ids)
    index = []
    rows = []
    for id in STASH_POS_COUNTS.keys():
        if id in ids or id.split('__')[0] in ids:
            index.append(id)
            rows.append(STASH_POS_COUNTS[id])
    return pd.DataFrame(rows, index=index).rename_axis('id').fillna(0).applymap(int)

In [28]:
df_meta = get_corpus_metadata()
ids_phil = df_meta[df_meta['discipline'] == 'Philosophy'].index.tolist()
ids_lit = df_meta[df_meta['discipline'] == 'Literature'].index.tolist()

In [29]:
def get_pos_word_counts(ids):
    ids=set(ids)
    all_feat2word2count = defaultdict(Counter)
    for id in STASH_FEAT2WORD2COUNT.keys():
        if id in ids or id.split('__')[0] in ids:
            for feat,word2count in STASH_FEAT2WORD2COUNT[id].items():
                all_feat2word2count[feat].update(word2count)
    return all_feat2word2count

# get_pos_word_counts(ids_phil)


In [30]:
def get_pos_word_egs(ids):
    ids=set(ids)
    all_feat2word2eg = defaultdict(dict)
    for id in STASH_FEAT2WORD2EG.keys():
        if id in ids or id.split('__')[0] in ids:
            for feat,word2eg in STASH_FEAT2WORD2EG[id].items():
                all_feat2word2eg[feat].update(word2eg)
    return all_feat2word2eg


In [61]:
# get_pos_word_egs(ids_lit)
g1,g1_ids='Philosophy',ids_phil
g2,g2_ids='Literature',ids_lit

df_pos_grp1 = get_pos_counts(g1_ids)
df_pos_grp2 = get_pos_counts(g2_ids)

df_pos = pd.concat([df_pos_grp1.assign(_target=g1), df_pos_grp2.assign(_target=g2)])
df_pos

,PRON,AUX,VERB,CCONJ,ADP,PUNCT,NOUN,ADV,PROPN,PART,...,advcl:relcl,nmod:unmarked,iobj,reparandum,csubj:outer,dislocated,orphan,csubj:pass,goeswith,_target
id,,,,,,,,,,,,,,,,,,,,,
phil/10.2307/20140633__02,89,66,100,34,116,99,190,47,19,26,...,0,0,0,0,0,0,0,0,0.0,Philosophy
phil/10.2307/43921200__10,8,7,18,46,86,258,225,7,196,7,...,0,0,0,0,0,0,0,0,0.0,Philosophy
phil/10.2307/26706722__04,38,76,78,21,110,121,215,64,18,19,...,0,0,0,0,0,0,0,0,0.0,Philosophy
phil/10.2307/2180906__06,46,51,88,30,135,138,199,44,40,19,...,0,0,0,0,0,0,0,0,0.0,Philosophy
phil/10.2307/27293790__02,59,66,78,23,99,162,207,55,17,23,...,0,0,0,0,0,0,0,0,0.0,Philosophy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
lit/436781__02,68,57,93,47,110,174,204,28,23,25,...,0,0,2,0,0,0,0,0,NaN,Literature
lit/27742966__06,37,28,91,27,113,131,201,47,54,43,...,1,0,1,0,0,0,0,0,NaN,Literature
lit/432521__02,51,44,92,41,117,109,195,43,11,18,...,1,2,0,1,0,0,0,0,NaN,Literature


In [60]:
words_grp1 = get_pos_word_counts(g1_ids)
words_grp2 = get_pos_word_counts(g2_ids)

egs_grp1 = get_pos_word_egs(g1_ids)
egs_grp2 = get_pos_word_egs(g2_ids)


In [45]:
# egs_grp1

In [46]:
df_means = df_pos.groupby('_target').mean()
feat2grp2mean = df_means.to_dict()
# feat2grp2mean

In [ ]:
FEAT_N = 100
FEAT_MIN_COUNT = 0


def get_egs(word2count, n = FEAT_N, min_count=FEAT_MIN_COUNT, word2eg={}):
    total = word2count.total()
    o = []
    for w,c in word2count.most_common():
        if n and len(o) >= n:
            break

        c = int(round(c/total*1000))
        if c >= min_count:
            # o.append(f'{w} ({c})')
            eg = word2eg.get(w)
            if eg:
                o.append(f'{w.upper()} ({c}) ["{eg}"]')
            else:
                o.append(f'{w} ({c})')
            n-=1
        else:
            break
        # o.append(w)
    return '; '.join(o)

In [59]:
pd.options.display.max_rows = 100
pd.options.display.max_colwidth = 200

ld = []
feat1 = None
for feat in feat2grp2mean:
    feat_d = {'feat':feat}
    grp2mean = feat2grp2mean[feat]
    
    feat_d[g1] = grp2mean[g1]
    feat_d[g2] = grp2mean[g2]
    feat_d[f'{g1} (top words)'] = get_egs(words_grp1[feat], n=FEAT_N, min_count=FEAT_MIN_COUNT)
    feat_d[f'{g2} (top words)'] = get_egs(words_grp2[feat], n=FEAT_N, min_count=FEAT_MIN_COUNT)

    feat_d[f'{g1} (examples)'] = get_egs(words_grp1[feat], n=FEAT_N, min_count=FEAT_MIN_COUNT, word2eg=egs_grp1[feat])
    feat_d[f'{g2} (examples)'] = get_egs(words_grp2[feat], n=FEAT_N, min_count=FEAT_MIN_COUNT, word2eg=egs_grp2[feat])
    
    # feat1 = feat
    ld.append(feat_d)
odf = pd.DataFrame(ld).dropna().set_index('feat')
diffcol = f'{g1}-{g2}'
odf[diffcol] = odf[g1] - odf[g2]
for c in odf:
    if not ' (' in c:
        odf[c] = odf[c].apply(round).apply(int).astype(int)
# odf = odf[['phil','lit','phil-lit','phil_egs','lit_egs','phil_egs2','lit_egs2']].sort_values('phil-lit', ascending=False)
# odf[['phil','lit','phil-lit', 'phil_egs']]
cols1=[g1,g2,diffcol]
odf = odf[cols1+[c for c in odf if not c in cols1]]
odf = odf.sort_values(diffcol, ascending=False)
odf


,Philosophy,Literature,Philosophy-Literature,Philosophy (top words),Literature (top words),Philosophy (examples),Literature (examples)
feat,,,,,,,
AUX,65,48,17,is (323); be (138); are (102); can (69); would (38); will (34); may (31); does (29); have (27); must (20); has (19); being (19); should (18); might (18); was (18); do (18); could (16); been (15); ...,is (295); be (99); are (83); was (75); have (47); can (43); has (36); would (32); been (30); may (29); were (28); will (25); had (24); does (20); must (18); could (16); being (14); might (14); sho...,"IS (323) [""complementation ± **is** induced by T""]; BE (138) [""but it can **be** provided with a""]; ARE (102) [""/ >v that **are** verifiable (or""]; CAN (69) [""way, but it **can** be provided with""...","IS (295) [""texture that **is** Vielleicht Esther""]; BE (99) [""topographies can **be** understood as knots""]; ARE (83) [""even if they **are** meant to overcome""]; WAS (75) [""whose Endstation **was*..."
mark,47,33,14,to (326); that (267); if (79); as (69); of (39); in (28); for (27); because (21); by (21); whether (20); since (14); while (11); though (10); although (7); than (7); without (7); so (4); from (4);...,to (376); that (203); as (99); if (47); of (37); in (30); for (24); because (22); by (22); while (16); though (15); since (11); than (11); whether (10); although (9); before (7); so (6); without (...,"TO (326) [""that are useful **to** compare the logical""]; THAT (267) [""Thirdly, we note **that** the elementary""]; IF (79) [""If** this new perspective""]; AS (69) [""definitions in CL, **as** follows...","TO (376) [""which asks us **to** position ourselves""]; THAT (203) [""Silverman emphasizes **that** as topographies""]; AS (99) [""in; as soon **as** we step into such""]; IF (47) [""If** the present and..."
SCONJ,31,21,11,that (407); if (120); as (105); of (55); in (37); for (36); because (32); by (31); whether (30); since (21); while (17); though (15); although (10); without (10); to (9); than (8); so (7); from (6...,that (336); as (163); if (78); of (59); in (43); because (36); by (35); for (33); while (26); though (24); since (18); whether (16); although (15); than (14); before (11); so (9); without (9); aft...,"THAT (407) [""general idea **that** some nonclassical""]; IF (120) [""objection vanishes **if** one accepts the""]; AS (105) [""definitions in CL, **as** follows""]; OF (55) [""defined as a class **of** ...","THAT (336) [""the impression **that** the various layers""]; AS (163) [""If, **as** argued before""]; IF (78) [""perpetrator, even **if** they are meant""]; OF (59) [""different way **of** handling Babi ..."
cop,28,19,9,is (590); be (159); are (157); being (29); was (24); were (15); been (9); am (8); 's (5); s (1); r (1); 're (1); 'm (0); m (0); isnt (0); re (0); does (0); thats (0); will (0); can (0); '( (0); im...,is (545); are (126); was (115); be (106); were (36); been (27); being (20); am (9); 's (8); 'd (1); 're (1); s (1); 'm (1); r (0); re (0); tis (0); m (0); ys (0); does (0); ist (0); wes (0); im (0...,"IS (590) [""following statement **is** then straightforward""]; BE (159) [""To **be** precise, we intend""]; ARE (157) [""statements that **are** useful to compare""]; BEING (29) [""responsible, but **be...","IS (545) [""is** state of complicity""]; ARE (126) [""whether or not we **are** personally related""]; WAS (115) [""sie nicht, **was** ich sehe""]; BE (106) [""Or it may **be** a vision of Sappho""]; WERE..."
NOUN,196,187,9,theory (8); way (7); case (7); sense (6); fact (6); knowledge (6); world (5); view (5); truth (5); question (5); belief (5); argument (5); time (4); reason (4); terms (4); point (4); example (4); ...,time (6); life (5); world (5); way (5); man (4); work (4); language (4); sense (4); poem (4); fact (4); text (4); words (3); self (3); history (3); form (3); part (3); play (3); nature (3); point ...,"THEORY (8) [""for a given **theory** T and classical""]; WAY (7) [""purely formal **way**, but it can be""]; 

In [53]:
# @title Most philosophy
odf.sort_values('phil-lit', ascending=False).head(10)

KeyError: 'phil-lit'

In [101]:
# @title Most lit
odf.sort_values('phil-lit', ascending=True).head(5)

,phil,lit,phil-lit,phil_egs,lit_egs,phil_egs2,lit_egs2
feat,,,,,,,
PROPN,21,43,-22,a (20); t (19); husserl (18); smith (17); god (15); kant (14); university (13); spinoza (11); descartes (10); jones (10); kripke (9); cambridge (9); davidson (9); aristotle (9); robin (8); l (7); sally (7); leibniz (7); amr (7); clark (7); press (6); press. (6); a. (6); c (6); schmidt (6),james (14); english (12); england (9); mr. (9); parlement (9); vivaldo (8); rimbaud (8); tophas (8); burney (8); jane (7); valery (7); wynnere (7); sir (7); rousseau (7); literature (7); hogg (7); eric (7); goldsmith (7); god (6); buffon (6); london (6); psalter (6); pocky (5); j. (5); orgilus (5),"A (20) [""simply X did **A** , we may hope""]; T (19) [""indeterminate whether **t** refers to a""]; HUSSERL (18) [""the former by **Husserl**, but it is also""]; SMITH (17) [""appreciate it, while **Smith** merely notices""]; GOD (15) [""through Christ to **God**, were taken over""]; KANT (14) [""concluded, that **Kant** was a Swedenborgian""]; UNIVERSITY (13) [""David E. Taylor **University** of Minnesota Picture""]; SPINOZA (11) [""from Descartes, **Spinoza**, Kant and Bohr""]; DESCARTES (10) [""elements from **Descartes**, Spinoza, Kant""]; JONES (10) [""same painting **Jones** may notice the""]; KRIPKE (9) [""reproduction of it, **Kripke** explicitly denies""]; CAMBRIDGE (9) [""Cambridge : **Cambridge** University Press""]; DAVIDSON (9) [""pump example by **Davidson** et al""]; ARISTOTLE (9) [""entirety, as **Aristotle** comprehended the""]; ROBIN (8) [""state, and that **Robin** will act rationally""]; L (7) [""which sentence of **L** would result in""]; SALLY (7) [""explanation why **Sally** used / despite""]; LEIBNIZ (7) [""Later, **Leibniz** explains how symbolism""]; AMR (7) [""the context of **AMR** is that automata""]; CLARK (7) [""as opposed to **Clark**, does not have""]; PRESS (6) [""Cambridge University **Press""]; PRESS. (6) [""Cambridge University **Press.** Verheij, B. Dialectical""]; A. (6) [""consistently include **A.** G — rdenfors has""]; C (6) [""then action **C**, and so on, just""]; SCHMIDT (6) [""Whereas **Schmidt** emphasizes the""]","JAMES (14) [""Yet **James** was not such an""]; ENGLISH (12) [""specialists in Old **English**, Henry Sweet""]; ENGLAND (9) [""Augustan Age of **England** ] is by Goldsmith""]; MR. (9) [""Mr.** MIuir presumably""]; PARLEMENT (9) [""whereas the **Parlement** poet employes it""]; VIVALDO (8) [""true purpose **Vivaldo** s be nevolence""]; RIMBAUD (8) [""sadistic Prince of **Rimbaud** s fairy - tale""]; TOPHAS (8) [""and fancy as **Tophas""]; BURNEY (8) [""Burney** satirizes a scenario""]; JANE (7) [""suggest that **Jane** Austen is being""]; VALERY (7) [""terminology used by **Valery** with that used""]; WYNNERE (7) [""The **Wynnere** poet, however""]; SIR (7) [""the relief of **Sir** John Moore s army""]; ROUSSEAU (7) [""lanceur of **Rousseau""]; LITERATURE (7) [""German V (Modem **Literature""]; HOGG (7) [""Hogg** s achievement""]; ERIC (7) [""suggested by **Eric** — as a risky""]; GOLDSMITH (7) [""not point to **Goldsmith""]; GOD (6) [""the power of **God**, he shall rise""]; BUFFON (6) [""headlines that **Buffon** had been the""]; LONDON (6) [""Luce of The **London** Prodigal plays""]; PSALTER (6) [""beginning of the **Psalter** was really like""]; POCKY (5) [""and the milk **Pocky""]; J. (5) [""J.** A. Without the""]; ORGILUS (5) [""Orgilus** s line of life""]"
case,104,123,-19,of (339); in (169); to (87); for (53); with (44); as (36); by (34); 's (33); on (32); from (28); at (25); about (15); between (12); into (8); than (7); within (7); like (6); upon (4); under (4); according (4); without (3); over (3); after (3); ' (3); against (3),of (343); in (162); to (75); 's (54); for (47); as (45); with (42); by (37); from (33); on (31); at (23); between (11); into (10); about (9); than (8); like (7); through (6); among (4); without (4); within (4); upon (4); over (3); against (3); out (3); under (3),"OF (339) [""